<a href="https://colab.research.google.com/github/brook-miller/2023mbai417/blob/main/2-homework/homework_2_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

this doesn't match up the closely matched company names but I think the rest of it is correct.  please take a look and let me know if you see any problems.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from datetime import datetime, timedelta
from dateutil.parser import parse

In [3]:
#@title creating dataframes for the subscription and the 60 months that we'll pull subscriptions for
df = pd.read_csv('https://raw.githubusercontent.com/brook-miller/2023mbai417/main/2-homework/subscription_info.csv')
df['date'] = pd.to_datetime(df['date_time']).dt.date
df = df.drop('date_time', axis=1)

from pandas.tseries.offsets import MonthEnd
months = pd.DataFrame()
months["first_of_month"] = pd.date_range('2017-01-01','2021-12-01', freq='MS').tolist()
months["end_of_month"] = months["first_of_month"] + MonthEnd(0)


In [4]:
#@title pulling down data from the API
import requests
from tqdm.notebook import trange, tqdm

ids = sorted(df["customer_id"].unique())

responses = []
for i in tqdm(range(0, len(ids), 500)):
  startat = ids[i]
  nosuccess = True
  while nosuccess:
    try:
      r = requests.get('https://jdw6paifodaa6hb7alta76h4te0erogq.lambda-url.us-west-2.on.aws/bulk?startid='+str(startat)+'&size=500', timeout=5)
      nosuccess = not(r.ok)
    except:
      nosuccess = True
  responses.append(r)

frames = []
for r in tqdm(responses):
  frames.append(pd.read_json(r.text))

dfcust = pd.concat(frames)
dfcust = dfcust.drop_duplicates()
dfcust

  0%|          | 0/503 [00:00<?, ?it/s]

  0%|          | 0/503 [00:00<?, ?it/s]

,id,email,name,company
0,2448,brentconway@mohawkindustries.com,Brent Conway,Mohawk Industries
1,2449,michaelpagejr.@jonesfinancial(edwardjones).com,Michael Page Jr.,Jones Financial (Edward Jones)
2,2450,jamesparrish@xeroxholdings.com,James Parrish,Xerox Holdings
3,2451,douglassnyder@opendoortechnologies.com,Douglas Snyder,Opendoor Technologies
4,2452,joshuaharris@appliedindustrialtechnologies.com,Joshua Harris,Applied Industrial Technologies
...,...,...,...,...
332,510553,michaellambert@lowe's.com,Michael Lambert,Lowe's
333,510580,ashleytownsend@tpg.com,Ashley Townsend,TPG
334,510647,aliciahunter@toro.com,Alicia Hunter,Toro
335,510970,markperry@appliedindustrialtechnologies.com,Mark Perry,Applied Industrial Technologies


In [5]:
#@title manipulating the original df using pivot to put subscribe, cancel, upgrade info into a single line
df2 = pd.pivot_table(df, values=["date","amount"], index=["customer_id"], aggfunc={"date": "first", "amount": "sum"}, columns=["action"]).reset_index()
df2.columns = df2.columns.map('_'.join).str.strip('_')

#plug cancel and upgrade with dates well outside of our analysis time frame
df2["date_cancel"] = df2["date_cancel"].fillna(datetime(2200,12,31).date())
df2["date_upgrade"] = df2["date_upgrade"].fillna(datetime(2200,12,31).date())
df2["amount_upgrade"] = df2["amount_upgrade"].fillna(0)

#with no info on the amount that the customers subscribed for I choose to drop.  Other assumptions are also valid
df2 = df2.dropna(subset = ["date_subscribe"])

df2 = df2.merge(dfcust[["id", "company"]], left_on="customer_id", right_on="id", how="left")
df2 = df2.drop("id", axis=1)
df2["churn_days"] = (df2["date_cancel"] - df2["date_subscribe"]).dt.days
df2

,customer_id,amount_cancel,amount_subscribe,amount_upgrade,date_cancel,date_subscribe,date_upgrade,company,churn_days
0,2448,NaN,49.99,0.00,2200-12-31,2017-01-01,2200-12-31,Mohawk Industries,67203
1,2449,NaN,49.99,0.00,2200-12-31,2017-01-01,2200-12-31,Jones Financial (Edward Jones),67203
2,2450,NaN,49.99,0.00,2200-12-31,2017-01-01,2200-12-31,Xerox Holdings,67203
3,2451,NaN,49.99,199.99,2200-12-31,2017-01-01,2017-03-31,Opendoor Technologies,67203
4,2452,NaN,99.99,0.00,2200-12-31,2017-01-01,2200-12-31,Applied Industrial Technologies,67203
...,...,...,...,...,...,...,...,...,...
122889,131970,NaN,49.99,0.00,2200-12-31,2021-12-31,2200-12-31,Acadia Healthcare,65378
122890,131971,NaN,49.99,0.00,2200-12-31,2021-12-31,2200-12-31,Renewable Energy Group,65378
122891,131972,NaN,49.99,0.00,2200-12-31,2021-12-31,2200-12-31,ArcBest,65378
122892,131973,NaN,49.99,0.00,2200-12-31,2021-12-31,2200-12-31,American Eagle Outfitters,65378


In [6]:
#@title cross join gives us a row for every single customer and all 60 months
customerRevMonths = df2.join(months, how="cross")

#we now have a dataset where each row has all the data for each customer as well as all the months in our time period
customerRevMonths

,customer_id,amount_cancel,amount_subscribe,amount_upgrade,date_cancel,date_subscribe,date_upgrade,company,churn_days,first_of_month,end_of_month
0,2448,NaN,49.99,0.0,2200-12-31,2017-01-01,2200-12-31,Mohawk Industries,67203,2017-01-01,2017-01-31
1,2448,NaN,49.99,0.0,2200-12-31,2017-01-01,2200-12-31,Mohawk Industries,67203,2017-02-01,2017-02-28
2,2448,NaN,49.99,0.0,2200-12-31,2017-01-01,2200-12-31,Mohawk Industries,67203,2017-03-01,2017-03-31
3,2448,NaN,49.99,0.0,2200-12-31,2017-01-01,2200-12-31,Mohawk Industries,67203,2017-04-01,2017-04-30
4,2448,NaN,49.99,0.0,2200-12-31,2017-01-01,2200-12-31,Mohawk Industries,67203,2017-05-01,2017-05-31
...,...,...,...,...,...,...,...,...,...,...,...
7373635,131974,NaN,49.99,0.0,2200-12-31,2021-12-31,2200-12-31,AvalonBay Communities,65378,2021-08-01,2021-08-31
7373636,131974,NaN,49.99,0.0,2200-12-31,2021-12-31,2200-12-31,AvalonBay Communities,65378,2021-09-01,2021-09-30
7373637,131974,NaN,49.99,0.0,2200-12-31,2021-12-31,2200-12-31,AvalonBay Communities,65378,2021-10-01,2021-10-31
7373638,131974,NaN,49.99,0.0,2200-12-31,2021-12-31,2200-12-31,AvalonBay Communities,65378,2021-11-01,2021-11-30


## In this section we keep only the rows in customerRevMonths which are valid based on their subscription dates

In [7]:
# keeping the months where the end of the month is after date_subscribe
customerRevMonths = customerRevMonths.loc[customerRevMonths["end_of_month"] >= customerRevMonths["date_subscribe"]]

customerRevMonths

,customer_id,amount_cancel,amount_subscribe,amount_upgrade,date_cancel,date_subscribe,date_upgrade,company,churn_days,first_of_month,end_of_month
0,2448,NaN,49.99,0.0,2200-12-31,2017-01-01,2200-12-31,Mohawk Industries,67203,2017-01-01,2017-01-31
1,2448,NaN,49.99,0.0,2200-12-31,2017-01-01,2200-12-31,Mohawk Industries,67203,2017-02-01,2017-02-28
2,2448,NaN,49.99,0.0,2200-12-31,2017-01-01,2200-12-31,Mohawk Industries,67203,2017-03-01,2017-03-31
3,2448,NaN,49.99,0.0,2200-12-31,2017-01-01,2200-12-31,Mohawk Industries,67203,2017-04-01,2017-04-30
4,2448,NaN,49.99,0.0,2200-12-31,2017-01-01,2200-12-31,Mohawk Industries,67203,2017-05-01,2017-05-31
...,...,...,...,...,...,...,...,...,...,...,...
7373399,131970,NaN,49.99,0.0,2200-12-31,2021-12-31,2200-12-31,Acadia Healthcare,65378,2021-12-01,2021-12-31
7373459,131971,NaN,49.99,0.0,2200-12-31,2021-12-31,2200-12-31,Renewable Energy Group,65378,2021-12-01,2021-12-31
7373519,131972,NaN,49.99,0.0,2200-12-31,2021-12-31,2200-12-31,ArcBest,65378,2021-12-01,2021-12-31
7373579,131973,NaN,49.99,0.0,2200-12-31,2021-12-31,2200-12-31,American Eagle Outfitters,65378,2021-12-01,2021-12-31


Here is the section where you will use previous examples to update the dataset

In [8]:
# keep the months where the first of the month is less than the date of cancel
# you can check your code by looking at the rows for a particular customer that canceled: customerRevMonths.loc[customerRevMonths["customer_id"] == idtocheck]
# add your code below




In [ ]:
# create a new column for "finalamount" and set it to the amount_subscribe
# add your code below



# where the end_of_month is greater than the date_upgrade set the finalamount to finalamount + amount_upgrade
# you can safely ignore the SettingWithCopyWarning
# add your code below



## If you completed the previous 2 cells correctly the rest of the cells will compute the metrics

In [ ]:
#yearly ARR
yearlyARR = customerRevMonths.groupby(customerRevMonths["first_of_month"].dt.year)["finalamount"].sum()
yearlyARR

In [ ]:
#churn calculation
df2["date_subscribe"] = pd.to_datetime(df2["date_subscribe"])
churndf = df2.loc[df2["churn_days"] < 366]
churn_count = churndf.groupby(churndf["date_subscribe"].dt.year)["customer_id"].count()
acquired_count = df2.groupby(df2["date_subscribe"].dt.year)["customer_id"].count()

rate_df = pd.DataFrame([churn_count, acquired_count]).transpose()
rate_df = rate_df.reset_index()
rate_df.columns = ["year", "churned_count", "acquired_count"]
rate_df["churn_rate"] = rate_df["churned_count"] / rate_df["acquired_count"]
rate_df

In [ ]:
# use the pivot_table function to develop a table that has companies and the date of the year
yearlyCompanyARR = customerRevMonths.pivot_table(values=["finalamount"], index="company", aggfunc="sum", columns=customerRevMonths["first_of_month"].dt.year).reset_index()
yearlyCompanyARR.columns = ["company", "2017", "2018", "2019", "2020", "2021"]
yearlyCompanyARR.sort_values(by="2020", ascending = False)

In [ ]:
#checking to see if there are companies that did not have revenue in a particular year
yearlyCompanyARR.isna().sum().sum()

In [ ]:
#there were 3 companies manually backing those out of the calculation
yearlyCompanyARR.sort_values(by="2017", ascending=True, na_position="first")
#2018 new customer revenue 1579.85 + 16567.83 + 549.89
# 
yearlyNRR = yearlyCompanyARR.sum(axis=0)
yearlyNRR = yearlyNRR.drop("company")
yearlyNRR = pd.DataFrame(yearlyNRR)
yearlyNRR.columns = ["revenue"]
yearlyNRR["year"] = pd.to_numeric(yearlyNRR.index)
yearlyNRR = yearlyNRR.reset_index(drop=True)
yearlyNRR["next"] = yearlyNRR["revenue"].shift(-1)
yearlyNRR.at[0, "next"] = 20131862.8 - (1579.85 + 16567.83 + 549.89)
yearlyNRR["nrr"] = yearlyNRR["next"] / yearlyNRR["revenue"]
yearlyNRR["year"] = yearlyNRR["year"] + 1
yearlyNRR = yearlyNRR.drop(["revenue","next"], axis=1)
yearlyNRR = yearlyNRR[:-1]
yearlyNRR

In the cell below provide brief (2-4 sentence) answers to the following questions:
1. What are the challenges of accessing data over APIs in order to do analysis?
2. In order to make data driven decisions, we often need to join together datasets. What are some of the challenges in joining together datasets from different sources?
3. What are examples of business logic / assumptions that could change / impact the analysis?
4. Company names work as an aggregation key for the company ARR and NRR analysis, could this lead to errors in analysis?
